In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
from operator import itemgetter
import statsmodels.api as sm


def is_majority_below_threshold(data, threshold):
    return (data < threshold).mean() > 0.5
def normalisation(series):
    return (series - series.min()) / (series.max() - series.min())



In [ ]:
data=pd.read_csv("e7-htr-currernt.csv")
fig = px.line(data,x="Timestamp",y="HT R Phase Current")
fig.show()


In [ ]:
fig.update_xaxes(range=[1, 2880])
fig.show()


We will be storing the individual data from each day into the dictionary daily_data

We will be creating the ideal day by taking the average of the best days as we can see from the graph in section 3 


In [ ]:
total_days=280
samples_per_day=288
daily_data={}
daily_data['Day_1']=data.iloc[0:222,1]
for d in range(total_days):
    start_index=d*samples_per_day+222
    end_index=start_index+samples_per_day
    day_data=data.iloc[start_index:end_index,1]
    daily_data[f'Day_{d+2}'] = day_data
    daily_data[f'Day_{d+2}']=daily_data[f'Day_{d+2}'].reset_index(drop=True)
pd.set_option('display.max_rows', 1000)


We will be finding the root mean square error of all of the data present in each day with the values of the ideal day

In [ ]:

best_days_data=[daily_data['Day_3'],daily_data['Day_7'],daily_data['Day_8'],daily_data['Day_9']]
ideal_day=(daily_data['Day_3']+daily_data['Day_9']+daily_data['Day_7']+daily_data['Day_8'])/4
rms_errors = {}
for day, data in daily_data.items():
    if day=='Day_1':
        continue
    rms_errors[day] = np.sqrt(np.sum((daily_data[day].iloc[79:217] - ideal_day[79:217]) ** 2))


We classify the data into good ,bad and missing

In [ ]:
classification={
    'good':[],
    'bad':[],
    'missing':[]
}
best_days_errors=[rms_errors['Day_3'],rms_errors['Day_7'],rms_errors['Day_8'],rms_errors['Day_9']]
mean=np.mean([rms_errors['Day_3'],rms_errors['Day_7'],rms_errors['Day_8'],rms_errors['Day_9']])

for day,data in daily_data.items():
    if day=='Day_1':
        continue
    if  is_majority_below_threshold(daily_data[day].iloc[119:155],1.1 ) or rms_errors[day]>(7.5)*np.amax(best_days_errors):
        classification['missing'].append(day)
    elif rms_errors[day]>0.8*np.amin(best_days_errors) and rms_errors[day]<15.8*np.amin(best_days_errors):
        classification['good'].append(day)
    else:
        classification['bad'].append(day)
    
num_good_days = len(classification['good'])
num_bad_days = len(classification['bad'])
num_missing_days = len(classification['missing'])

print(f"Number of Good Days: {num_good_days}")
print(f"Number of Bad Days: {num_bad_days}")
print(f"Number of Missing Days: {num_missing_days}")


Now we group the good days data into a single dataframe and improve the quality by removing outliers and replacing them with ideal day values for the specific datapoint

In [ ]:
df_good_initial=pd.concat([daily_data[day] for day in classification['good']], axis=0).reset_index(drop=True)
SSE=0
n=0
for day in classification['good']:
    for i in range(288):
        SSE=SSE+(daily_data[day].iloc[i] - ideal_day[i])**2
        n=n+1
s=(SSE/n-2)**0.5
for day in classification['good']:
    for i in range(288):
        if (daily_data[day].iloc[i] - ideal_day[i])**2>2*s:
            daily_data[day].iloc[i] = ideal_day[i]
        else:
            continue
            


In [ ]:
df=pd.concat([daily_data[day] for day in daily_data.keys()],axis=0).reset_index(drop=True)
datacpy=pd.read_csv("e7-htr-currernt.csv")
fig = px.line(x=df.index,y=datacpy["HT R Phase Current"].iloc[0:80862],title="Good Data After Improvement(zoom in)")
fig.add_scatter(x=df.index,y=df,fillcolor="black")
fig.show()

Now we go ahead with the mlr model for the data ,adding hours, minutes and their higher and mutual powers as our features

In [ ]:
df_good = pd.concat([daily_data[day] for day in classification['good']], axis=0).reset_index(drop=True)

feature_names = {f'hours^{i}': [] for i in range(1, 5)}
feature_names.update({f'minutes^{i}': [] for i in range(1, 5)})

for h1 in range(1, 5):
    for m1 in range(1, 5):
        feature_names[f'hours^{h1}*minutes^{m1}'] = []

print(feature_names)

h=0
m=0
for p in range(80862):
    total_minutes = p * 5  
    h=(total_minutes // 60) % 24 
    m=total_minutes % 60 
    for q in range(1, 5):
        feature_names[f'hours^{q}'].append(h ** q)
        feature_names[f'minutes^{q}'].append(m ** q)
       
    for h1 in range(1, 5):
        for m1 in range(1, 5):
            feature_names[f'hours^{h1}*minutes^{m1}'].append((h ** h1) * (m ** m1))
features_df = pd.DataFrame(feature_names)
features_df = normalisation(features_df)




now we train our model with 80% of the good day data and eliminate the features with a p value over 0.05

In [ ]:

total_rows = len(df_good)
train_end_index = int(total_rows * 0.8)

train_data_y = df_good[:train_end_index]
train_data_x=features_df[:train_end_index]
train_data_y=train_data_y.astype(float)
train_data_x=train_data_x.astype(float)

mlr_question = sm.OLS(train_data_y,train_data_x).fit()
p_values=mlr_question.pvalues
eliminated_features = p_values[p_values > 0.05].index
train_data_x_new = train_data_x.drop(columns=eliminated_features)

mlr_2 = sm.OLS(train_data_y,train_data_x_new).fit()
coefficients=mlr_2.params
p_values_2=mlr_2.pvalues
R_squared_2=mlr_2.rsquared

eliminated_features_2 = p_values_2[p_values_2 > 0.05].index
train_data_x_final = train_data_x_new.drop(columns=eliminated_features_2)
mlr_final = sm.OLS(train_data_y,train_data_x_final).fit()

coefficients=mlr_final.params
p_values_final=mlr_final.pvalues
R_squared_final=mlr_final.rsquared
F_value_final=mlr_final.fvalue
coefficients=mlr_final.params
print(mlr_final.summary())


We predict the next 20% good day data and find the mean square error with the actual values

In [ ]:
validation_data_y = df_good[train_end_index:]
validation_data_x1 = features_df[train_end_index:train_end_index + len(validation_data_y)]
validation_data_x2=validation_data_x1.drop(columns=eliminated_features_2)
validation_data_x=validation_data_x2.drop(columns=eliminated_features)
y_predict=mlr_final.predict(validation_data_x)
y_predict = y_predict.apply(lambda x: max(x, 0.1))
mse_test_data=np.mean(np.square(y_predict-validation_data_y))
print(mse_test_data)


Now we use our model to replace the bad and missing days with values predicted by our model

In [ ]:
bad_missing_data = pd.concat([daily_data[day] for day in classification['bad']+classification['missing']], axis=0).reset_index(drop=True)
bad_missing_features = features_df.loc[:].drop(columns=eliminated_features)
y_predict_bad_missing = mlr_final.predict(bad_missing_features).reset_index(drop=True)
y_predict_bad_missing=y_predict_bad_missing.apply(lambda x: max(x,0.1))
y_predict_bad_missing=y_predict_bad_missing[:len(bad_missing_data)]
fig = px.line(x=bad_missing_data.index,y=bad_missing_data,title='OLD VS NEW MISSING AND BAD DATA(zoom to see overlap and changes)')
fig.add_scatter(x=bad_missing_data.index,y=y_predict_bad_missing)
fig.show()

In [ ]:
print("BAD AND MISSING DAYS INITIAL: ")
print(bad_missing_data.describe())
print("BAD AND MISSING DAYS FINAL: ")
print(y_predict_bad_missing.describe())



In [ ]:
print("GOOD DAYS INITIAL: ")
print(df_good_initial.describe())
print("GOOD DAYS FINAL: ")
print(df_good.describe())